In [10]:
import dash
import dash_html_components as html
from dash.dependencies import Input, Output
from markupsafe import Markup
from dash import dcc
import plotly.express as px
import pandas as pd
import numpy as np
from faker import Faker
import plotly.graph_objs as go
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.utilities import JsCode
from folium.features import GeoJsonPopup
from folium.plugins import TimeSliderChoropleth
from folium import plugins
from folium.plugins import HeatMap

In [11]:
# Set random seed for reproducibility
np.random.seed(0)

# Initialize Faker to generate fake data
fake = Faker()

# Define the number of rows in the dataset
num_rows = 15000

# Create a DataFrame to store the fake dataset
practice_df = pd.DataFrame()

# Create opening hour variable
opening_hour_range = range(7, 9)
opening_hour = np.random.choice(opening_hour_range, size=num_rows)

# Create closing hour variable
closing_hour_range = range(17, 22)
closing_hour = np.random.choice(closing_hour_range, size=num_rows)

# Add the opening hour variable to the DataFrame
practice_df['opening_hour'] = opening_hour

# Add the closing hour variable to the DataFrame
practice_df['closing_hour'] = closing_hour

# Generate aed location variable
min_lat, max_lat = 50.5, 50.6  # Latitude boundaries of Belgium
min_lon, max_lon = 3, 4   # Longitude boundaries of Belgium
practice_df['Latitude'] = np.random.uniform(min_lat, max_lat, size=num_rows)
practice_df['Longitude'] = np.random.uniform(min_lon, max_lon, size=num_rows)

aed_placement_categories = ['outside', 'inside', 'difficult']
aed_placement = np.random.choice(aed_placement_categories, size=num_rows)
practice_df['aed_placement'] = aed_placement

# Define buffer distances
buffer_distances = {
    'outside': 200,  # ~200 meters in degrees
    'inside': 150,   # ~150 meters in degrees
    'difficult': 125  # ~125 meters in degrees
}

practice_df['buffer_distance'] = practice_df['aed_placement'].map(buffer_distances)

# Add unique identifier for each row
practice_df['id'] = practice_df.index

# Convert DataFrame to GeoDataFrame
practice_gdf = gpd.GeoDataFrame(
    practice_df,
    geometry=[Point(xy) for xy in zip(practice_df['Longitude'], practice_df['Latitude'])],
    crs="EPSG:4326"  # WGS84 coordinate system
)

# Pre-calculate colors based on hours
for hour in range(24):
    condition = (
        (practice_df['opening_hour'] <= hour) & 
        (practice_df['closing_hour'] >= hour)
    )
    practice_df[f'color_{hour}'] = np.where(condition, 'white', 'black')

def generate_folium_map(selected_hour):
    color_column = f'color_{selected_hour}'
    practice_gdf['color'] = practice_df[color_column]

    m = folium.Map(location=[50.5, 4.3517], zoom_start=8)
    for idx, row in practice_gdf.iterrows():
        folium.Circle(
            location=(row['Latitude'], row['Longitude']),
            radius=row['buffer_distance'],
            color=row['color'],
            fill=True,
            fill_opacity=1,  # Add fill opacity for better visualization
            stroke=False
        ).add_to(m)
    return m._repr_html_()

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Slider(
        id='time-slider',
        min=0,
        max=23,
        step=1,
        value=12,
        marks={i: str(i) for i in range(24)}
    ),
    html.Iframe(
        id='folium-map',
        srcDoc=generate_folium_map(12),  # Generate initial map for hour 12
        width='100%',
        height='600px'
    )
])

@app.callback(
    Output('folium-map', 'srcDoc'),
    Input('time-slider', 'value')
)
def update_map(selected_hour):
    return generate_folium_map(selected_hour)

if __name__ == '__main__':
    app.run_server(debug=True)

In [12]:
print(practice_df.head())

   opening_hour  closing_hour   Latitude  Longitude aed_placement  \
0             7            17  50.566809   3.957612       outside   
1             8            20  50.533628   3.764495     difficult   
2             8            18  50.578681   3.731273       outside   
3             7            17  50.503064   3.004353       outside   
4             8            19  50.522428   3.641763     difficult   

   buffer_distance  id                  geometry color_0 color_1  ...  \
0              200   0  POINT (3.95761 50.56681)   black   black  ...   
1              125   1  POINT (3.76450 50.53363)   black   black  ...   
2              200   2  POINT (3.73127 50.57868)   black   black  ...   
3              200   3  POINT (3.00435 50.50306)   black   black  ...   
4              125   4  POINT (3.64176 50.52243)   black   black  ...   

  color_15 color_16 color_17 color_18 color_19 color_20 color_21 color_22  \
0    white    white    white    black    black    black    black    b